In [1]:
import numpy as np
import awkward as ak
import uproot, json

# from util.matrix import loadMatrices, baseTransform
from pathlib import Path

np.set_printoptions(precision=6)
np.set_printoptions(suppress=True)


In [2]:
def loadMatrices(filename):
    with open(filename) as f:
        result = json.load(f)
    for key, value in result.items():
        result[key] = np.array(value).reshape(4, 4)
    return result

def saveMatrices(matrices, fileName):

    # create path if needed
    if not Path(fileName).parent.exists():
        Path(fileName).parent.mkdir()

    # warn if overwriting
    if Path(fileName).exists():
        print(f"WARNING. Replacing file: {fileName}!\n")
        Path(fileName).unlink()

    # flatten matrices, make a copy (pass-by-reference!)
    saveMatrices = {}
    for p in matrices:
        saveMatrices[p] = np.ndarray.tolist(np.ndarray.flatten(matrices[p]))

    with open(fileName, "w") as f:
        json.dump(saveMatrices, f, indent=2, sort_keys=True)

def baseTransform(mat, matFromAtoB):
    """
    Reminder: the way this works is that the matrix pointing from pnd to sen0 transforms a matrix IN sen0 back to Pnd
    If you want to transform a matrix from Pnd to sen0, and you have the matrix to sen0, then you need to give
    this function inv(matTo0). I know it's confusing, but that's the way this works.

    Example: matrixInPanda = baseTransform(matrixInSensor, matrixPandaToSensor)
    """
    return matFromAtoB @ mat @ np.linalg.inv(matFromAtoB)

In [3]:
def best_fit_transform(A, B):
    """
    Calculates the least-squares best-fit transform that maps corresponding points A to B in m spatial dimensions
    Input:
      A: Nxm numpy array of corresponding points
      B: Nxm numpy array of corresponding points
    Returns:
      T: (m+1)x(m+1) homogeneous transformation matrix that maps A on to B
      R: mxm rotation matrix
      t: mx1 translation vector
    """

    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    H = np.dot(AA.T, BB)
    U, _, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
        Vt[m - 1, :] *= -1
        R = np.dot(Vt.T, U.T)

    # translation
    t = centroid_B.T - np.dot(R, centroid_A.T)

    # homogeneous transformation
    T = np.identity(m + 1)
    T[:m, :m] = R
    T[:m, m] = t

    return T, R, t

In [4]:
# these are detector parameters. they can only change with a redesign of the detector,
# so there is no point in putting these in a config file.
availableOverlapIDs = range(7)
availableModuleIDs = range(40)

idealDetectorMatrices = loadMatrices("../config/detectorMatricesIdeal.json")


In [5]:
#! Attention! Are they sorted? I don't think so!

# read from root and sort to numpy, process one root file ENTTIRELY at a time an then write all np arrays to disk
# only then read next file (saves on IO)

# TODO: we don't actually need to store distance. It's already been calculated by the pair finder for the static cut and the dynamic cut requires the 3d center of mass, so the 1d distance is useless anyway

path = "/mnt/himsterData/roklasen/LumiFit/LMD-15.00-dkohUogm/data/reco_uncut/aligned-alignment-matrices/"
rootFileWildcard = "Lumi_Pairs_*.root:pndsim"

runIndex = 0
maxNoOfFiles = 50
npyOutputDir = "../temp/npPairs"


if not Path(npyOutputDir).exists():
    Path(npyOutputDir).mkdir(parents=True)

    for arrays in uproot.iterate(
        path+rootFileWildcard,
        [
            "PndLmdHitPair._moduleID",
            "PndLmdHitPair._overlapID",
            "PndLmdHitPair._hit1",
            "PndLmdHitPair._hit2",
        ],
        # library="np", # DONT use numpy yet, we need the awkward array for the TVector3
        allow_missing=True,  # some files may be empty, skip those):
    ):
        runIndex += 1

        # some evvents have no hits, but thats not a problem
        # after the arrays are flattened, those empty events
        # simply disappear
        moduleIDs = np.array(ak.flatten(arrays["PndLmdHitPair._moduleID"]))
        overlapIDs = np.array(ak.flatten(arrays["PndLmdHitPair._overlapID"]))
        hit1x = ak.flatten(arrays["PndLmdHitPair._hit1"].fX)
        hit1y = ak.flatten(arrays["PndLmdHitPair._hit1"].fY)
        hit1z = ak.flatten(arrays["PndLmdHitPair._hit1"].fZ)
        hit2x = ak.flatten(arrays["PndLmdHitPair._hit2"].fX)
        hit2y = ak.flatten(arrays["PndLmdHitPair._hit2"].fY)
        hit2z = ak.flatten(arrays["PndLmdHitPair._hit2"].fZ)

        hit1 = np.array((hit1x, hit1y, hit1z)).T
        hit2 = np.array((hit2x, hit2y, hit2z)).T

        distVec = np.linalg.norm(hit1 - hit2, axis=1)

        arr = np.array((moduleIDs, hit1x, hit1y, hit1z, hit2x, hit2y, hit2z, overlapIDs)).T

        for moduleID in availableModuleIDs:
            mask = arr[:, 0] == moduleID
            thisOverlapsArray = arr[mask][:, 1:]

            # read array from disk
            fileName = f"{npyOutputDir}/pairs-modID-{moduleID}.npy"

            try:
                oldContent = np.load(fileName)
            # first run, file not already present
            except FileNotFoundError:
                oldContent = np.empty((0, 7))

            # merge
            newContent = np.concatenate((oldContent, thisOverlapsArray))

            # write back to disk
            np.save(file=fileName, arr=newContent, allow_pickle=False)

        if runIndex == maxNoOfFiles:
            break


In [6]:
def dynamicCut(hitPairs, cutPercent=2):

    if cutPercent == 0:
        return hitPairs

    # calculate center of mass of differences
    dRaw = hitPairs[:, 3:6] - hitPairs[:, :3]
    com = np.average(dRaw, axis=0)

    # shift newhit2 by com of differences
    newhit2 = hitPairs[:, 3:6] - com

    # calculate new distance for cut
    dRaw = newhit2 - hitPairs[:, :3]
    newDist = np.power(dRaw[:, 0], 2) + np.power(dRaw[:, 1], 2)

    # sort by distance and cut some percent from end (discard outliers)
    cut = int(len(hitPairs) * cutPercent / 100.0)
    # sort by new distance
    hitPairs = hitPairs[newDist.argsort()]
    # cut off largest distances, NOT lowest
    hitPairs = hitPairs[:-cut]

    return hitPairs


In [7]:
# find matrices for ONE overlap

use2D = True

with open('../config/moduleIDtoModulePath.json') as f:
    moduleIdToModulePath = json.load(f)

def findMatrix(PairData, thisModule):

    # apply dynamic cut
    PairData = dynamicCut(PairData, 2)

    # if idealOverlapInfos is None or PairData is None:
    #     raise Exception(f"Error! Please load ideal detector matrices and numpy pairs!")

    # if len(idealDetectorMatrices) < 1:
    #     raise Exception("ERROR! Please set ideal detector matrices!")

    # Make C a homogeneous representation of hits1 and hits2
    hit1H = np.ones((len(PairData), 4))
    hit1H[:, 0:3] = PairData[:, :3]

    hit2H = np.ones((len(PairData), 4))
    hit2H[:, 0:3] = PairData[:, 3:6]

    # Attention! Always transform to module-local system,
    # otherwise numerical errors will make the ICP matrices unusable!
    # (because z is at 11m, while x is 30cm and y is 0)
    # also, we're ignoring z distance, which we can not do if we're in
    # PND global, due to the 40mrad rotation.
    transformToLocalSensor = True
    if transformToLocalSensor:
        icpDimension = 2
        # get matrix lmd to module
        modulePath = moduleIdToModulePath[str(thisModule)]
        matToModule = idealDetectorMatrices[modulePath]

        # invert to transform pairs from lmd to sensor
        toModInv = np.linalg.inv(matToModule)

        # Transform vectors (remember, C and D are vectors of vectors = matrices!)
        hit1T = np.matmul(toModInv, hit1H.T).T
        hit2T = np.matmul(toModInv, hit2H.T).T

    else:
        print("WARNING! ICP working in Panda global, NOT sensor local.")
        print("This will likely produce wrong overlap matrices,")
        print("If the hit points are not transformed beforehand!")
        hit1T = hit1H
        hit2T = hit2H

    if use2D:
        icpDimension = 2
        # make 2D versions for ICP
        A = hit1T[:, :2]
        B = hit2T[:, :2]
    else:
        icpDimension = 3
        # make 3D versions for ICP
        A = hit1T[:, :3]
        B = hit2T[:, :3]

    # find ideal transformation
    T, _, _ = best_fit_transform(A, B)

    # print(f'this is T for {moduleIdToModulePath[str(thisModule)]}:\n{T}')

    # copy 3x3 Matrix to 4x4 Matrix
    if icpDimension == 2:
        M = np.identity(4)
        M[:2, :2] = T[:2, :2]
        M[:2, 3] = T[:2, 2]
        thisOverlapMatrix = M

    elif icpDimension == 3:
        thisOverlapMatrix = T

    # transformResultToPND = True
    if transformToLocalSensor:
        # remember, matToModule goes from Pnd->Module
        # base trafo is T A T^-1,
        # T = Pnd->Module

        # TODO: use generalized base transformation function
        thisOverlapMatrix = (
            (matToModule) @ thisOverlapMatrix @ np.linalg.inv(matToModule)
        )
    return thisOverlapMatrix


In [8]:
# we're now looping over the module IDs, not the overlap IDs.
# that means we need a new loop here and filter the npy files again
# for the small overlap ID. the previous overlapID doesn't exists anymore

# this is a nested dict so that we can access overlapMatrices[moduleID][overlapID] == overlapMatrix
overlapMatrices = {}

for moduleID in availableModuleIDs:

    pairsOnModule = np.load(f"{npyOutputDir}/pairs-modID-{moduleID}.npy")

    # print(f"Processing module {moduleID}, loading file {npyOutputDir}/pairs-modID-{moduleID}.npy")

    overlapMatrices[str(moduleID)] = {}
    for overlapID in availableOverlapIDs:

        # mask for overlapID
        mask = pairsOnModule[:, 6] == overlapID

        # ignore distance, we don't need it anymore
        pairsOnOverlap = pairsOnModule[mask][:, :6]

        # make dict with overlap matrices
        overlapMatrices[str(moduleID)][str(overlapID)] = findMatrix(
            pairsOnOverlap, moduleID
        )

In [9]:
from numpy.linalg import inv

"""
Author: R. Klasen, roklasen@uni-mainz.de or r.klasen@gsi.de or r.klasen@ep1.rub.de

Uses multiple overlap matrices and the ideal detector matrices to compute alignment matrices for each sensor.
Each combiner is responsible for a single module.

It requires the misalignment matrices of sensor 0 and 1 for its assigned module.
We will obtain these with microscopic measurements.

The overlap matrices come from the Reco Points and are thus already in PANDA global. All calculations must
happen in PANDA GLOBAL, but the actual misalignment matrices are applied in the current super frame.
That means the misalignment matrix for sensor 1 is applied in the frame of sensor 1, NOT PANDA GLOBAL.
(Doesn't matter how it was generated, this is the way it's done by FAIRROOT, and that's by design).
For the math to work out, all matrices must be transformed to PANDA GLOBAL before calculations.
To compare the found alignment matrices with the input misalignment matrces, the found alignment
matrices must be transformed to the reference frame of the corresponding sensor.
"""


class alignmentMatrixCombiner:
    def __init__(self, moduleID):
        self.moduleID = moduleID
        self.modulePath = moduleIdToModulePath[str(self.moduleID)]
        self.alignmentMatrices = {}
        self.overlapMatrices = None
        self.idealDetectorMatrices = None
        self.externalMatrices = None

    def setOverlapMatrices(self, matrices):
        self.overlapMatrices = matrices

    def setIdealDetectorMatrices(self, matrices):
        self.idealDetectorMatrices = matrices

    def setExternallyMeasuredMatrices(self, matrices):
        self.externalMatrices = matrices

    def getAlignmentMatrices(self):
        return self.alignmentMatrices

    def initCalculator(self):
        # required paths and ideal matrices for base transformations
        p0 = self.modulePath + "/sensor_0"
        p1 = self.modulePath + "/sensor_1"
        p2 = self.modulePath + "/sensor_2"
        p3 = self.modulePath + "/sensor_3"
        p4 = self.modulePath + "/sensor_4"
        p5 = self.modulePath + "/sensor_5"
        p6 = self.modulePath + "/sensor_6"
        p7 = self.modulePath + "/sensor_7"
        self.m0 = self.idealDetectorMatrices[p0]
        self.m1 = self.idealDetectorMatrices[p1]
        self.m2 = self.idealDetectorMatrices[p2]
        self.m3 = self.idealDetectorMatrices[p3]
        self.m4 = self.idealDetectorMatrices[p4]
        self.m5 = self.idealDetectorMatrices[p5]
        self.m6 = self.idealDetectorMatrices[p6]
        self.m7 = self.idealDetectorMatrices[p7]

        # attention! m0star and m1star are applied by FAIRROOT in the system
        # of sensors 0 and 1 respectively
        # that means m1star here is given in sensor 1, not PANDA GLOBAL
        # for the math to work out, m1star must be transformed to PANDA GLOBAL
        m0starInSensor0 = self.externalMatrices[p0]
        m1starInSensor1 = self.externalMatrices[p1]
        self.m0star = baseTransform(m0starInSensor0, self.m0)
        self.m1star = baseTransform(m1starInSensor1, self.m1)

        # see sensorIDs.pdf
        self.mICP0t4 = self.overlapMatrices[str(self.moduleID)]["0"]
        self.mICP1t7 = self.overlapMatrices[str(self.moduleID)]["1"]
        self.mICP2t7 = self.overlapMatrices[str(self.moduleID)]["2"]
        self.mICP3t7 = self.overlapMatrices[str(self.moduleID)]["3"]
        self.mICP3t5 = self.overlapMatrices[str(self.moduleID)]["4"]
        self.mICP3t6 = self.overlapMatrices[str(self.moduleID)]["5"]
        # not used since it's so small
        self.mICP1t5 = self.overlapMatrices[str(self.moduleID)]["6"]

    def combine1to2(self):

        # see Equation 7.23 in my PhD thesis
        mBstar = self.m1star @ inv(inv(self.mICP2t7) @ self.mICP1t7)

        # now, transform mBstar from PANDA GLOBAL into the system of sensorB
        # (because thats where it was applied by FAIRROOT)
        mBstarInSensorB = baseTransform(mBstar, inv(self.m2))

        return mBstarInSensorB

    def combine1to3(self):

        # see Equation 7.23 in my PhD thesis
        mBstar = self.m1star @ inv(inv(self.mICP3t7) @ self.mICP1t7)

        # now, transform mBstar from PANDA GLOBAL into the system of sensorB
        # (because thats where it was applied by FAIRROOT)
        mBstarInSensorB = baseTransform(mBstar, inv(self.m3))

        return mBstarInSensorB

    def combine0to4(self):

        # see Equation 7.23 in my PhD thesis
        mBstar = self.m0star @ inv(self.mICP0t4)

        # now, transform mBstar from PANDA GLOBAL into the system of sensorB
        # (because thats where it was applied by FAIRROOT)
        mBstarInSensorB = baseTransform(mBstar, inv(self.m4))

        return mBstarInSensorB

    def combine1to5(self):

        # see Equation 7.23 in my PhD thesis
        mBstar = self.m1star @ inv(self.mICP3t5 @ inv(self.mICP3t7) @ self.mICP1t7)

        # now, transform mBstar from PANDA GLOBAL into the system of sensorB
        # (because thats where it was applied by FAIRROOT)
        mBstarInSensorB = baseTransform(mBstar, inv(self.m5))

        return mBstarInSensorB

    def combine1to6(self):

        # see Equation 7.23 in my PhD thesis
        mBstar = self.m1star @ inv(self.mICP3t6 @ inv(self.mICP3t7) @ self.mICP1t7)

        # now, transform mBstar from PANDA GLOBAL into the system of sensorB
        # (because thats where it was applied by FAIRROOT)
        mBstarInSensorB = baseTransform(mBstar, inv(self.m6))

        return mBstarInSensorB

    def combine1to7(self):

        # see Equation 7.23 in my PhD thesis
        mBstar = self.m1star @ inv(self.mICP1t7)

        # now, transform mBstar from PANDA GLOBAL into the system of sensorB
        # (because thats where it was applied by FAIRROOT)
        mBstarInSensorB = baseTransform(mBstar, inv(self.m7))

        return mBstarInSensorB

    def combineMatrices(self):
        # checks here
        if self.overlapMatrices is None or self.idealDetectorMatrices is None:
            raise Exception(
                f"ERROR! Please set overlaps, overlap matrices and ideal detector matrices first!"
            )

        if self.externalMatrices is None:
            self.externalMatrices = {}
            print("------------------- ATTENTION !!! -------------------")
            print("No external matrices set! Assuming identity matrices!")
            print("Restuls in wrong matrices if detector is mialigned!!!")
            print("------------------- ATTENTION !!! -------------------")
            self.externalMatrices[self.modulePath + "/sensor_0"] = np.eye(4)
            self.externalMatrices[self.modulePath + "/sensor_1"] = np.eye(4)


        self.initCalculator()

        mat2mis = self.combine1to2()
        mat3mis = self.combine1to3()
        mat4mis = self.combine0to4()
        mat5mis = self.combine1to5()
        mat6mis = self.combine1to6()
        mat7mis = self.combine1to7()

        # now, all the overlap misalignments are in PND global!

        # copy the given misalignments, so that all are in the final set!
        self.alignmentMatrices[self.modulePath + "/sensor_0"] = self.externalMatrices[
            self.modulePath + "/sensor_0"
        ]
        self.alignmentMatrices[self.modulePath + "/sensor_1"] = self.externalMatrices[
            self.modulePath + "/sensor_1"
        ]

        # store computed misalignment matrices to internal dict
        self.alignmentMatrices[self.modulePath + "/sensor_2"] = mat2mis
        self.alignmentMatrices[self.modulePath + "/sensor_3"] = mat3mis
        self.alignmentMatrices[self.modulePath + "/sensor_4"] = mat4mis
        self.alignmentMatrices[self.modulePath + "/sensor_5"] = mat5mis
        self.alignmentMatrices[self.modulePath + "/sensor_6"] = mat6mis
        self.alignmentMatrices[self.modulePath + "/sensor_7"] = mat7mis

        print(
            f"successfully computed misalignments on module {self.modulePath} for {len(self.alignmentMatrices)} sensors!"
        )


In [10]:
sensorAlignMatrices = {}
externalMatrices = loadMatrices("../matrices/100u-case-1/externalMatrices-sensors.json")

for moduleID in availableModuleIDs:
    combiner = alignmentMatrixCombiner(moduleID)
    combiner.setIdealDetectorMatrices(idealDetectorMatrices)
    combiner.setOverlapMatrices(overlapMatrices)
    combiner.setExternallyMeasuredMatrices(externalMatrices)
    combiner.combineMatrices()
    sensorAlignMatrices.update(combiner.getAlignmentMatrices())


successfully computed misalignments on module /cave_1/lmd_root_0/half_0/plane_0/module_0 for 8 sensors!
successfully computed misalignments on module /cave_1/lmd_root_0/half_0/plane_0/module_1 for 8 sensors!
successfully computed misalignments on module /cave_1/lmd_root_0/half_0/plane_0/module_2 for 8 sensors!
successfully computed misalignments on module /cave_1/lmd_root_0/half_0/plane_0/module_3 for 8 sensors!
successfully computed misalignments on module /cave_1/lmd_root_0/half_0/plane_0/module_4 for 8 sensors!
successfully computed misalignments on module /cave_1/lmd_root_0/half_1/plane_0/module_0 for 8 sensors!
successfully computed misalignments on module /cave_1/lmd_root_0/half_1/plane_0/module_1 for 8 sensors!
successfully computed misalignments on module /cave_1/lmd_root_0/half_1/plane_0/module_2 for 8 sensors!
successfully computed misalignments on module /cave_1/lmd_root_0/half_1/plane_0/module_3 for 8 sensors!
successfully computed misalignments on module /cave_1/lmd_root_0

In [11]:
# dump matrices to json
saveMatrices(sensorAlignMatrices, "../matrices/100u-case-1/sensorAlignmentMatrices.json")

In [12]:
testSensor = "/cave_1/lmd_root_0/half_0/plane_2/module_3/sensor_4"

mMat = loadMatrices("../matrices/100u-case-1/misMat-sensors.json")


print("-------------------- this time --------------------")
print(sensorAlignMatrices[testSensor] * 1e4)
print("-------------------- actual --------------------")
print(mMat[testSensor] * 1e4)

print("-------------------- residuals matrix --------------------")
print( ((sensorAlignMatrices[testSensor]) @ inv(mMat[testSensor] ))*1e4  )


-------------------- this time --------------------
[[ 9999.840012   -56.56624     -0.         -87.835257]
 [   56.56624   9999.840012     0.         -21.682441]
 [    0.          -0.       10000.           0.      ]
 [    0.           0.           0.       10000.      ]]
-------------------- actual --------------------
[[ 9999.836821   -57.127443     0.         -90.921921]
 [   57.127443  9999.836821     0.         -22.489409]
 [    0.           0.       10000.           0.      ]
 [    0.           0.           0.       10000.      ]]
-------------------- residuals matrix --------------------
[[ 9999.999984     0.561212    -0.           3.087926]
 [   -0.561212  9999.999984     0.           0.801866]
 [    0.          -0.       10000.           0.      ]
 [    0.           0.           0.       10000.      ]]
